In [43]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from tradingUtils import *
from indicadores import *
import labeling as lb
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)

Loading BokehJS ...

In [44]:
DATA = prio_data
YEAR = None

In [45]:
# Função a ser executada: faz o backtesting para um dado modelo e ano
# Se "year" for passado, ele faz o backtest nesse ano
# Se não, ele faz nos últimos 365 dias de dado
def backtesting_model(olhc, model, year = None, **kwargs):
    # Calculando os indicadores
    indicators = agg_indicators(olhc)
    indicators = normalize_indicators(indicators)
    # Calculando o rótulo
    y = np.array(lb.labelData(olhc, 0.1)).ravel()
    # Eliminando as linhas com NaN
    indicators["y"] = y
    indicators = indicators.dropna()
    # Separando os dados de treino e backtest com base no ano selecionado
    # Se tiver um ano específico, separa ele para backtest
    if year:
        indicators_train = indicators[indicators.index.year != year]
        indicators_backtest = indicators[indicators.index.year == year]
    # Se não, pega os últimos 365 dias
    else:
        last_day = indicators.index[-1]
        indicators_train = indicators[indicators.index <= last_day - pd.DateOffset(years = 1)]
        indicators_backtest = indicators[indicators.index > last_day - pd.DateOffset(years = 1)]
    # Convertendo para numpy arrays, caso ainda não estejam
    X = np.array(indicators_train)[:, :-1]
    y = np.array(indicators_train)[:, -1]
    # Treinando o modelo
    model = model(X, y, **kwargs)
    # Predizendo a política para aquele ano
    pred = model.predict(np.array(indicators_backtest)[:, :-1])
    # Salvando a predição em um dataframe adequado (próximas 4 linhas)
    # Pegando os dados originais do período de backtest
    if year:
        olhc_backtest = olhc[olhc.index.year != year]
    else:
        last_day = olhc.index[-1]
        olhc_backtest = olhc[olhc.index > last_day - pd.DateOffset(years = 1)]
    # Criando uma série com a predição e o index do ano
    policy = pd.Series(pred, index = olhc_backtest.index)
    # Colocando a predição nesse dataframe
    olhc_backtest["Signal"] = 0
    olhc_backtest.loc[policy.index, "Signal"] = policy

    return olhc_backtest

## Fazendo a Rede Neural

In [55]:
def mlp(data, y, hidden_layers=(100, 100, 100), activation='logistic', 
        solver='adam', max_iter=500, random_state=42):
    """
    Treina um modelo de rede neural MLP e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame contendo os dados de entrada.
    - hidden_layers: Tupla com o tamanho das camadas ocultas.
    - activation: Função de ativação a ser usada.
    - solver: Algoritmo de otimização a ser usado.
    - max_iter: Número máximo de iterações.
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_mlp: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    # Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

    # Definindo a rede neural com múltiplas camadas
    mlp = MLPClassifier(hidden_layer_sizes = hidden_layers,
                        activation = activation,
                        solver = solver,
                        max_iter = max_iter,
                        random_state = random_state)

    # Treina a rede neural
    mlp.fit(X_train, y_train)

    # Faz previsões de classe
    y_pred_mlp = mlp.predict(X_test)

    # Exibe o relatório de classificação para o MLP
    report = classification_report(y_test, y_pred_mlp)
    print(report)

    return mlp

In [ ]:
# Testando o modelo
dados_mlp = backtesting_model(DATA, mlp, year = YEAR)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


              precision    recall  f1-score   support

        -1.0       0.52      0.60      0.56       299
         0.0       0.00      0.00      0.00        34
         1.0       0.51      0.44      0.48       290

    accuracy                           0.50       623
   macro avg       0.34      0.35      0.34       623
weighted avg       0.48      0.50      0.49       623



c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pedro\AppData\Local\Temp\ipykernel_11440\330422410.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olhc_backtest["Signal"] = 0


In [57]:
bt = Backtest(dados_mlp, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2023-08-22 00:00:00
End                       2024-08-21 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                        98.8
Equity Final [$]                  7796.807688
Equity Peak [$]                  10236.528915
Return [%]                         -22.031923
Buy & Hold Return [%]                3.880266
Return (Ann.) [%]                     -22.187
Volatility (Ann.) [%]               23.204979
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -48.534991
Avg. Drawdown [%]                  -25.206201
Max. Drawdown Duration      352 days 00:00:00
Avg. Drawdown Duration      178 days 00:00:00
# Trades                                   56
Win Rate [%]                        46.428571
Best Trade [%]                       8.743895
Worst Trade [%]                    -10.772406
Avg. Trade [%]                    

## Fazendo o modelo de Random Forest

In [46]:
def random_forest(data, y, n_estimators=100, max_depth=None, random_state=42):
    """
    Treina um modelo de Random Forest e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame com as features e a variável alvo.
    - target_column: Nome da coluna alvo no DataFrame.
    - n_estimators: Número de árvores na floresta.
    - max_depth: Profundidade máxima das árvores (None para ilimitado).
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_rf: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    # Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

    # Definindo o modelo Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                 random_state=random_state)

    # Treina o modelo Random Forest
    rf.fit(X_train, y_train)

    # Faz previsões de classe
    y_pred_rf = rf.predict(X_test)

    # Exibe o relatório de classificação para o Random Forest
    report = classification_report(y_test, y_pred_rf)
    print(report)

    return rf

In [47]:
# Testando o modelo
dados_rf = backtesting_model(DATA, random_forest, year = YEAR)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


              precision    recall  f1-score   support

        -1.0       0.50      0.56      0.53       299
         0.0       0.10      0.03      0.05        34
         1.0       0.49      0.47      0.48       290

    accuracy                           0.49       623
   macro avg       0.36      0.35      0.35       623
weighted avg       0.47      0.49      0.48       623



C:\Users\pedro\AppData\Local\Temp\ipykernel_20224\330422410.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olhc_backtest["Signal"] = 0


In [55]:
bt = Backtest(dados_rf, OurStrategy, cash=10000)
stats = bt.run()
print(stats["Equity Final [$]"])

# # Exibindo o resultado
# bt.plot()
# print(stats)

7206.859725999993


## Fazendo o Modelo Gradient Boosting

In [61]:
def gradient_boosting(data, y, random_state=42):
    """
    Treina um modelo de Gradient Boosting e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame com as features e a variável alvo.
    - target_column: Nome da coluna alvo no DataFrame.
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_gb: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    # Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

    # Definindo o modelo Gradient Boosting
    gb_model = GradientBoostingClassifier(random_state=random_state)

    # Treina o modelo Gradient Boosting
    gb_model.fit(X_train, y_train)

    # Faz previsões de classe
    y_pred_gb = gb_model.predict(X_test)

    # Exibe o relatório de classificação para o Gradient Boosting
    report = classification_report(y_test, y_pred_gb)
    print(report)

    return gb_model

In [ ]:
dados_gb = backtesting_model(DATA, gradient_boosting, year = YEAR)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


              precision    recall  f1-score   support

        -1.0       0.51      0.57      0.54       299
         0.0       0.25      0.09      0.13        34
         1.0       0.49      0.47      0.48       290

    accuracy                           0.50       623
   macro avg       0.42      0.38      0.38       623
weighted avg       0.49      0.50      0.49       623



C:\Users\pedro\AppData\Local\Temp\ipykernel_11440\330422410.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olhc_backtest["Signal"] = 0


In [63]:
bt = Backtest(dados_gb, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2023-08-22 00:00:00
End                       2024-08-21 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                        99.2
Equity Final [$]                 17659.498415
Equity Peak [$]                  17659.498415
Return [%]                          76.594984
Buy & Hold Return [%]                3.880266
Return (Ann.) [%]                   77.400235
Volatility (Ann.) [%]               54.591711
Sharpe Ratio                         1.417802
Sortino Ratio                        4.049136
Calmar Ratio                         5.481846
Max. Drawdown [%]                  -14.119373
Avg. Drawdown [%]                   -4.117846
Max. Drawdown Duration       78 days 00:00:00
Avg. Drawdown Duration       15 days 00:00:00
# Trades                                   32
Win Rate [%]                           71.875
Best Trade [%]                       12.56229
Worst Trade [%]                      -5.45455
Avg. Trade [%]                    